### Importing the modules

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

### Load the data

In [2]:
df = pd.read_csv("/content/covid_toy.csv")

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
df["cough"].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df["city"].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [7]:
df["gender"].value_counts()

Female    59
Male      41
Name: gender, dtype: int64

### Train Test Split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(df.drop("has_covid",axis=1),df["has_covid"],random_state=2,test_size=0.2)

In [9]:
X_train.head()

,age,gender,fever,cough,city
35,82,Female,102.0,Strong,Bangalore
11,65,Female,98.0,Mild,Mumbai
84,69,Female,98.0,Strong,Mumbai
44,20,Male,102.0,Strong,Delhi
73,34,Male,98.0,Strong,Kolkata


### Perform Simple Imputer On Fever Column

In [10]:
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[["fever"]])
X_test_fever = si.transform(X_test[["fever"]])

In [11]:
X_train_fever.shape

(80, 1)

### Perform Ordinal Encoder On Cough Column as it is Ordinal Feature

In [12]:
oe = OrdinalEncoder(categories=[["Mild","Strong"]])

X_train_cough = oe.fit_transform(X_train[["cough"]])
X_test_cough = oe.transform(X_test[["cough"]])

In [13]:
X_train_cough.shape

(80, 1)

### Perform One Hot Encoding on Gender and City as it is Nominal feature

In [14]:
ohe = OneHotEncoder(drop="first",sparse=False)

X_train_gender_city = ohe.fit_transform(X_train[["gender","city"]])
X_test_gender_city = ohe.transform(X_test[["gender","city"]])

In [15]:
X_train_gender_city.shape

(80, 4)

### Extract Age Feature and Convert it into array

In [16]:
X_train_age = X_train.drop(columns=["gender","fever","cough","city"]).values
X_test_age = X_test.drop(columns=["gender","fever","cough","city"]).values

In [17]:
X_train_age.shape

(80, 1)

In [18]:
X_train_age.shape

(80, 1)

### Concatenate all the columns

In [19]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

In [20]:
X_train_transformed.shape

(80, 7)

In [21]:
X_test_transformed.shape

(20, 7)

### Using Column Transformer

In [22]:
X_train,X_test,y_train,y_test = train_test_split(df.drop("has_covid",axis=1),df["has_covid"],random_state=2,test_size=0.2)

In [23]:
from sklearn.compose import ColumnTransformer

In [24]:
transformer = ColumnTransformer(transformers=[
    ("tnf1",SimpleImputer(),["fever"]),
    ("tnf2",OrdinalEncoder(categories=[["Mild","Strong"]]),["cough"]),
    ("tnf3",OneHotEncoder(sparse=False,drop="first"),["gender","city"])
],remainder="passthrough")

In [25]:
transformer.fit_transform(X_train).shape

(80, 7)

In [26]:
transformer.transform(X_test).shape

(20, 7)